# Summary of This Baseline

* Using tiling method based on https://www.kaggle.com/iafoss/panda-16x128x128-tiles
    * Simply setting the `N = 36` and `sz=256` then extract from median resolution
* Create 6x6 big image from 36 tiles
* Efficientnet-B0
* Binning label
    * E.g.
        * `label = [0,0,0,0,0]` means `isup_grade = 0`
        * `label = [1,1,1,0,0]` means `isup_grade = 3`
        * `label = [1,1,1,1,1]` means `isup_grade = 5`
* BCE loss
* Augmentation on both tile level and big image level
* CosineAnnealingLR for one round

In [1]:
!pip install git+https://github.com/ildoonet/pytorch-gradual-warmup-lr.git

  Cloning https://github.com/ildoonet/pytorch-gradual-warmup-lr.git to /tmp/pip-req-build-gl2k0cl_
  Running command git clone -q https://github.com/ildoonet/pytorch-gradual-warmup-lr.git /tmp/pip-req-build-gl2k0cl_
  Created wheel for warmup-scheduler: filename=warmup_scheduler-0.3.2-py3-none-any.whl size=3881 sha256=2b0c556991fb526e8b52253704e5f9c6b1443cf3d7208f36c4bc3876ada54774
  Stored in directory: /tmp/pip-ephem-wheel-cache-8kaks6ct/wheels/bf/81/52/0e3bc0b645a339f94c76b4dcb8c8b7a5f588a614f5add83b9f
Successfully built warmup-scheduler


In [2]:
!pip install efficientnet-pytorch

In [3]:
DEBUG = False

In [4]:
from warmup_scheduler import GradualWarmupScheduler

In [5]:
import os
import sys
import gc
import random
from zipfile import ZipFile
from PIL import Image, ImageChops
import warnings
warnings.simplefilter('ignore', Image.DecompressionBombWarning)

import time
import skimage.io
import numpy as np
import pandas as pd
import cv2
import PIL.Image
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import StepLR, ExponentialLR
from torch.optim.sgd import SGD
from torch import FloatTensor
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler, SequentialSampler

from efficientnet_pytorch import EfficientNet

import albumentations as A
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score
from sklearn.utils import shuffle

In [6]:
Image.MAX_IMAGE_PIXELS = None

# Config

In [7]:
df = pd.read_csv('train.csv')

# shuffle
df = shuffle(df)

# split
train_df = df[df.split != 0]
train_df = train_df.sample(1000) if DEBUG else train_df
val_df = df[df.split == 0]
n = len(val_df) // 2
test_df = val_df.iloc[:n]
val_df = val_df.iloc[n:]


len(train_df), len(val_df), len(test_df)

(9289, 664, 663)

In [8]:
train_df.head()

,image_id,data_provider,isup_grade,gleason_score,split,folder
2552,283b3a40795ae1992f0ddd56a55c396a,karolinska,1,3+3,2,3
1552,66ed3b7961aea86a4a44a6798884b6bc,radboud,4,4+4,4,1
8438,e2c1421a3794ba9dc1673c6509628485,karolinska,0,0+0,7,1
1654,5ee927ed5d354f5f72e5d341276eb5be,radboud,3,4+3,4,1
8664,ed9a38a9f1dfa898e536cd91f13d98b1,radboud,3,4+3,4,3


In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [10]:
GRAY_TR = 235
N_TR = 0.85
WHITE_TR = 0.95

class config:
    SZ = 256    # not used, just to declare start point, 
    N = 36      # not used, just to declare start point, 
    LEVEL = 1   # not used, just to declare start point, 
    
    IMG_SIZE = 1536
    BS = 2 # batch size
    NUM_WORKERS = 4
    CLASSES = 5
    BASE = 'efficientnet-b0'
    LR = 3e-4 # learning rate
    EPOCHS = 1 if DEBUG else 10
    SEED = 2020

In [11]:
# seed

def seed_everything(seed=2020):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(config.SEED)

In [12]:
torch.cuda.empty_cache()
gc.collect()

88

# Model

In [13]:
class MyModel(nn.Module):
    def __init__(self, backbone, out_dim):
        super(MyModel, self).__init__()
        self.enet = EfficientNet.from_name(backbone)
        self.myfc = nn.Linear(self.enet._fc.in_features, out_dim)
        self.enet._fc = nn.Identity()

    def extract(self, x):
        return self.enet(x)

    def forward(self, x):
        x = self.extract(x)
        x = self.myfc(x)
        return x

In [14]:
# test model
if DEBUG:
    
    model = MyModel('efficientnet-b0', 5)
    x = FloatTensor(np.random.randn(5, 3, 512, 512))
    y = model(x)
    print(y.shape)

    del model, x, y

# Dataset

In [15]:
def get_tiles(img, SZ=256, N=36):
    '''
    dynamically select SZ and N
    img: numpy array since this image has been processed by select_level, remove_gray, crop, and transpose
    Return 
        tiles, SZ, N
    '''
    
    # pad img
    h, w, c = img.shape
    pad_h = (SZ - h % SZ) % SZ 
    pad_w = (SZ - w % SZ) % SZ 

    img2 = np.pad(img, [[pad_h // 2, pad_h - pad_h // 2], 
                        [pad_w // 2, pad_w - pad_w//2], 
                        [0,0]], constant_values=255)
    
    # choose tiles
    img3 = img2.reshape(
        img2.shape[0] // SZ,
        SZ,
        img2.shape[1] // SZ,
        SZ,
        3
    )
    
    new_row, new_col = img3.shape[0], img3.shape[2]
    img3 = img3.transpose(0, 2, 1, 3, 4).reshape(-1, SZ, SZ, 3) # (783, 256, 256, 3)
    info = (img3.reshape(img3.shape[0],-1).sum(-1) < WHITE_TR * SZ*SZ*3*255).sum() # how many tiles are not white
    
    # get new N
    possible_N = int(np.sqrt(info))**2
    if N < possible_N:
        N = possible_N
        
    idxs = np.argsort(img3.reshape(img3.shape[0],-1).sum(-1))[:N]
    tiles = img3[idxs]
        
    return tiles, SZ, N, info

In [16]:
def find_tiles(img, SZ=256, N=36):
    tiles, SZ, N, info = get_tiles(img, SZ=SZ, N=N)

    
    # too much white tiles
    # smallest SZ is 64
    while info <= int(N_TR * N) and SZ > 64:
        SZ = SZ // 2
        tiles, SZ, N, info = get_tiles(img, SZ=SZ, N=N)
        
        
    # pad 
    # for example 32 < 30(0.85*36) 
    if tiles.shape[0] < N:
        tiles = np.pad(tiles, [ [0, N-len(tiles)], [0,0],[0,0],[0,0]], constant_values=255)
        
        
    return tiles, SZ, N, info

In [17]:
train_transforms = A.Compose([A.Transpose(p=0.5), 
                             A.VerticalFlip(p=0.5),
                             A.HorizontalFlip(p=0.5)])
val_transforms = A.Compose([])

In [18]:
class MyTrainDataset(Dataset):
    def __init__(self, df, split='train', shuffle_df=False, shuffle_tiles=False):
        super().__init__()
        
        if shuffle_df:
            df = shuffle(df)
        self.df = df.reset_index(drop=True)
        
        self.split = split
        self.shuffle_tiles = shuffle_tiles
            
        
    def __len__(self):
        return len(self.df)
    
    
    def __getitem__(self, idx):
        
        # read img
        name = self.df.image_id[idx] 
        folder = self.df.folder[idx]
        myzip = ZipFile(f'pandacroppedlevel1{folder}.zip')
        path = f'train{folder}/{name}.jpeg'
        img = myzip.open(path)
        img = Image.open(img)
        img = np.array(img)  
        
        
        tiles, SZ, N, info = find_tiles(img) # find suitable tiles, SZ, N
        

        # apply transform to each img
        imgs = []
        for t in tiles:
            if self.split == 'train':
                t_aug = train_transforms(**{'image': t})['image']
            elif self.split == 'val':
                t_aug = val_transforms(**{'image': t})['image']
                
            imgs.append(t_aug) 
        
        
        # shuffle tiles
        if self.shuffle_tiles:
            imgs = shuffle(imgs)
     
        # concat
        n = int(np.sqrt(N)) # new Z
        images = np.zeros((SZ*n, SZ*n, 3), dtype=np.int32) # new SZ
        for i in range(n):
            for j in range(n):
                images[i*SZ : (i+1)*SZ, j*SZ : (j+1)*SZ, :] = imgs[i*n+j]
        
            
        # normalize 
        images = 255 - images # reverse 
        if self.split == 'train':
            images = train_transforms(image=images)['image']
        images = A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])(image=images)['image']
        
        # resize 
        images = cv2.resize(images, (config.IMG_SIZE, config.IMG_SIZE))
        images = torch.tensor(images).permute(2, 0, 1)
        
        # ordinal regression
        label = np.zeros(5).astype(np.float32)
        origin_label = self.df.isup_grade[idx]
        label[:origin_label] = 1.
        label = torch.tensor(label)
        
        return images, label

In [19]:
# test get_tiles
if DEBUG:
    ds = MyTrainDataset(df, shuffle_tiles=True)  
    
    # show samples
    fig, ax = plt.subplots(5, 5, figsize=(20, 20))
    for i in range(5):
        for j in range(5):  
            x, y = ds[i*5+j]
            ax[i, j].imshow(x.permute(1, 2, 0)*0.5+0.5)
            ax[i, j].set_title(y)

# Train & Val

In [20]:
def train_on(dl, optimizer):

    # 1 clean
    torch.cuda.empty_cache()
    gc.collect()
    
    model.train()
    
    # 3 analysis
    loss_epoch = []
    y_preds_epoch = []
    y_epoch = []
    
    bar = tqdm(dl, total=len(dl))
    for (x, y) in bar:
        
        x, y = x.to(device), y.to(device)
        
        y_preds = model(x)
        loss = nn.BCEWithLogitsLoss()(y_preds, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        
        # cohen
        y_preds_np = y_preds.sigmoid().sum(1).detach().cpu().round()
        y_np = y.sum(1).detach().cpu().numpy()
        cohen = cohen_kappa_score(y_preds_np, y_np, weights='quadratic')
            
            
        # add
        loss_np = loss.detach().cpu().numpy()
        
        loss_epoch.append(loss_np)
        y_epoch.append(y_np)
        y_preds_epoch.append(y_preds_np)
        
        bar.set_description('loss: %.4f, cohen: %.4f' % (loss_np, cohen))
        
        
        # clean
        del x, y, y_preds, loss, loss_np, y_preds_np, y_np, cohen
        torch.cuda.empty_cache()
        gc.collect()
        
    # get train cohen
    y_epoch = np.concatenate(y_epoch)
    y_preds_epoch = np.concatenate(y_preds_epoch)
    train_cohen = cohen_kappa_score(y_preds_epoch, y_epoch, weights='quadratic')

    return np.mean(loss_epoch), train_cohen

In [21]:
def val_on(dl):

    # 1 clean
    torch.cuda.empty_cache()
    gc.collect()
    
    model.eval()
    
    # 3 analysis
    loss_epoch = []
    y_epoch = []
    y_preds_epoch = []

    bar = tqdm(dl, total=len(dl))
    with torch.no_grad():
        for (x, y) in bar:
            x, y = x.to(device), y.to(device)
            y_preds = model(x)

            loss = nn.BCEWithLogitsLoss()(y_preds, y)
            

            # get cohen
            y_preds_np = y_preds.sigmoid().sum(1).detach().cpu().numpy().round()
            y_np = y.sum(1).detach().cpu().numpy()
            cohen = cohen_kappa_score(y_preds_np, y_np, weights='quadratic')
            
            bar.set_description('Loss: %.4f, cohen: %.4f' % (loss, cohen))
            
            # add
            y_preds_epoch.append(y_preds_np)
            y_epoch.append(y_np)
            
            loss_np = loss.detach().cpu().numpy()
            loss_epoch.append(loss_np)
            
            # clean
            del y, y_preds, y_np, y_preds_np, loss, loss_np
            torch.cuda.empty_cache()
            gc.collect()
            

    # get val cohen
    y_epoch = np.concatenate(y_epoch)
    y_preds_epoch = np.concatenate(y_preds_epoch)
    val_cohen = cohen_kappa_score(y_preds_epoch, y_epoch, weights='quadratic')
    
#     val_cohen_k = cohen_kappa_score(y_preds_epoch[val_df['data_provider'] == 'karolinska'], val_df[val_df['data_provider'] == 'karolinska'].isup_grade.values, weights='quadratic')
#     val_cohen_r = cohen_kappa_score(y_preds_epoch[val_df['data_provider'] == 'radboud'], val_df[val_df['data_provider'] == 'radboud'].isup_grade.values, weights='quadratic')
#     print('val_cohen', val_cohen, 'val_cohen_k', val_cohen_k, 'val_cohen_r', val_cohen_r)

 
    return np.mean(loss_epoch), val_cohen

    

# Create Dataloader & Model & Optimizer

In [22]:
def get_data(fold):

    train_ds = MyTrainDataset(train_df, split='train', shuffle_tiles=True)  
    val_ds = MyTrainDataset(val_df, split='val', shuffle_tiles=True)  

    train_dl = DataLoader(train_ds, 
                          batch_size=config.BS, 
                          sampler=RandomSampler(train_df), 
                          num_workers=config.NUM_WORKERS,
                          drop_last = True)
    val_dl = DataLoader(val_ds, 
                        batch_size=config.BS, 
                        sampler=SequentialSampler(val_df), 
                        num_workers=config.NUM_WORKERS,
                        drop_last=True)
    
    print(len(train_ds), len(val_ds))
    return train_dl, val_dl

# Run Training

In [23]:
torch.cuda.empty_cache()
gc.collect()

110

In [24]:
# important for resuming training since lr would change

def save_model(path):
    ck = {'state_dict': model.state_dict(), 
          'optim': optimizer.state_dict()}
    torch.save(ck, path)  

In [25]:
def load_model(path):
    ck = torch.load(path)
    model.load_state_dict(ck['state_dict'])
    optimizer.load_state_dict(ck['optim'])
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, config.EPOCHS)

In [26]:
best_cohen = 0.8640

In [27]:
train_dl, val_dl = get_data(0)

9289 664


In [28]:
model = MyModel(config.BASE, out_dim=config.CLASSES)
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=config.LR)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, config.EPOCHS)

In [29]:
def train(epochs):
    global best_cohen
    
    for epoch in range(1, epochs+1):
        print(time.ctime(), 'Epoch:', epoch)

        scheduler.step(epoch-1)

        
        train_loss, train_cohen = train_on(train_dl, optimizer)
        print(f'Epoch: {epoch}, train_loss: {train_loss:.4f}, train_cohen: {train_cohen:.4f}')

        
        val_loss, val_cohen = val_on(val_dl)
        print(f'Epoch: {epoch}, val_loss: {val_loss:.4f}, val_cohen: {val_cohen:.4f}')

        
        content = time.ctime() + ' ' + f'Epoch {epoch}, lr: {optimizer.param_groups[0]["lr"]:.7f}, train loss: {train_loss:.4f}, train cohen: {(train_cohen):.4f} val loss: {val_loss:.4f}, cohen: {(val_cohen):.4f}'
        with open(f'efnb0_log.txt', 'a') as appender:
            appender.write(content + '\n')

        # to avoid machine crash
        save_model(f'efnb0-{epoch}.pth')

        # best cohen
        if val_cohen > best_cohen:
            print('Save model')
            save_model(f'efnb0-best.pth')
            best_cohen = val_cohen


    save_model(f'efnb0_final.pth')

In [32]:
load_model('efnb0-3.pth')

In [35]:
val_on(val_dl)

(0.2529182, 0.8896315733484628)

In [31]:
train(60)

Sat Jul  4 05:55:30 2020 Epoch: 1


/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:620: RuntimeWarning: invalid value encountered in true_divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)



Epoch: 1, train_loss: 0.1557, train_cohen: 0.9166



Epoch: 1, val_loss: 0.2884, val_cohen: 0.8771
Save model
Sat Jul  4 07:04:16 2020 Epoch: 2


...


Epoch: 12, train_loss: 0.0705, train_cohen: 0.9735



Epoch: 12, val_loss: 0.3380, val_cohen: 0.8854
Sat Jul  4 19:39:43 2020 Epoch: 13



Epoch: 13, train_loss: 0.0739, train_cohen: 0.9715



Epoch: 13, val_loss: 0.3683, val_cohen: 0.8762
Sat Jul  4 20:48:30 2020 Epoch: 14


KeyboardInterrupt: 